In [1]:
from web3 import Web3
from eth_account import Account
import time

# === 基本配置 ===
PRIVATE_KEY = "6125280318b5f59e2864716ea224dec13c1aeecc7c8747b882c1a4b190e2e2a1"
ADDRESS = Web3.to_checksum_address("0x6722Da2471fd315205ae9d9664bd5171A5ba2Abc")
web3 = Web3(Web3.HTTPProvider("https://ethereum-sepolia.publicnode.com"))
print("1")
# === WETH 合约地址和 ABI ===
WETH = web3.to_checksum_address("0xfff9976782d46cc05630d1f6ebab18b2324d6b14")
WETH_ABI = [{
    "inputs": [],
    "name": "deposit",
    "outputs": [],
    "stateMutability": "payable",
    "type": "function"
}]
contract = web3.eth.contract(address=WETH, abi=WETH_ABI)
print("2")
# === 构造交易 ===
tx = contract.functions.deposit().build_transaction({
    "from": ADDRESS,
    "value": Web3.to_wei(1.2, 'ether'),
    "gas": 180_000,
    "gasPrice": web3.to_wei("100", "gwei"),
    "nonce": web3.eth.get_transaction_count(ADDRESS),
    "chainId": 11155111
})
print("3")
# === 签名并发送 ===
signed_tx = web3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)

print("交易发送成功！Tx Hash:", tx_hash.hex())
print("等待上链中...")

# === 等待确认 ===
receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)
print("已上链！交易状态:", receipt["status"])


1
2
3
交易发送成功！Tx Hash: b1981ebfa4e25e33e49351aa543174e82b87b3bb46724bdd480f87ed138f60e0
等待上链中...


TimeExhausted: Transaction HexBytes('0xb1981ebfa4e25e33e49351aa543174e82b87b3bb46724bdd480f87ed138f60e0') is not in the chain after 120 seconds

In [12]:
from web3 import Web3
from eth_account import Account

# === 基本配置 ===
PRIVATE_KEY = "6125280318b5f59e2864716ea224dec13c1aeecc7c8747b882c1a4b190e2e2a1"
ADDRESS = Web3.to_checksum_address("0x6722Da2471fd315205ae9d9664bd5171A5ba2Abc")
web3 = Web3(Web3.HTTPProvider("https://ethereum-sepolia.publicnode.com"))

# === 合约地址 ===
WETH = Web3.to_checksum_address("0xfff9976782d46cc05630d1f6ebab18b2324d6b14")
COWSWAP_CONTRACT = Web3.to_checksum_address("0x9008d19f58aabd9ed0d60971565aa8510560ab41")

# === approve ABI ===
WETH_ABI = [{
    "constant": False,
    "inputs": [
        {"name": "spender", "type": "address"},
        {"name": "amount", "type": "uint256"}
    ],
    "name": "approve",
    "outputs": [{"name": "", "type": "bool"}],
    "type": "function"
}]

# === 实例化合约 ===
contract = web3.eth.contract(address=WETH, abi=WETH_ABI)

# === 构建交易 ===
amount = Web3.to_wei(1.2, "ether")  # ✅ 授权最多可用多少 WETH
nonce = web3.eth.get_transaction_count(ADDRESS)

tx = contract.functions.approve(COWSWAP_CONTRACT, amount).build_transaction({
    "from": ADDRESS,
    "gas": 100_000,
    "gasPrice": web3.to_wei("30", "gwei"),
    "nonce": nonce,
    "chainId": 11155111
})

# === 签名并发送 ===
signed = Account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = web3.eth.send_raw_transaction(signed.raw_transaction)

print("Approve 交易发送成功！Tx Hash:", tx_hash.hex())
print("等待上链中...")

receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=180)
print("已上链！交易状态:", receipt["status"])



Approve 交易发送成功！Tx Hash: d6d44bfb557439252113373266b19c55a145302a1a1132f8cafe16c45cde96f7
等待上链中...
已上链！交易状态: 1


In [3]:
from web3 import Web3
from eth_account import Account
import json
import requests
import binascii

# === 配置 ===
PRIVATE_KEY = "6125280318b5f59e2864716ea224dec13c1aeecc7c8747b882c1a4b190e2e2a1"
ADDRESS = Web3.to_checksum_address("0x6722Da2471fd315205ae9d9664bd5171A5ba2Abc")
web3 = Web3(Web3.HTTPProvider("https://ethereum-sepolia.publicnode.com"))

# === Token 合约 ===
WETH = Web3.to_checksum_address("0xfff9976782d46cc05630d1f6ebab18b2324d6b14")
COW = Web3.to_checksum_address("0x91d7F3C37f67B067E207bCcb7d5b50dF6102CeCb")
COWSWAP_CONTRACT = Web3.to_checksum_address("0x9008d19f58aabd9ed0d60971565aa8510560ab41")

# === 构造订单 ===
app_data = "{\"version\":\"0.9.0\",\"metadata\":{}}"
app_data_hash = Web3.keccak(text=app_data).hex()

current_time = int(web3.eth.get_block('latest').timestamp)
valid_to = current_time + 600  # 10分钟后过期

order = {
    "sellToken": WETH,
    "buyToken": COW,
    "receiver": ADDRESS,
    "sellAmount": str(Web3.to_wei(0.5, "ether")),
    "buyAmount": "0",  # 市价单
    "validTo": valid_to,
    "appData": app_data,
    "appDataHash": app_data_hash,
    "feeAmount": "0",
    "kind": "sell",
    "partiallyFillable": False,
    "sellTokenBalance": "erc20",
    "buyTokenBalance": "erc20",
    "signingScheme": "eip1271",  # 改用eip1271签名方案
    "quoteId": 0,
    "from": ADDRESS.lower()
}

# === 使用CowSwap API获取订单摘要 ===
print("正在获取订单摘要...")
quote_response = requests.post(
    "https://api.cow.fi/sepolia/api/v1/quote",
    json={
        "sellToken": WETH,
        "buyToken": COW,
        "receiver": ADDRESS,
        "sellAmountBeforeFee": str(Web3.to_wei(0.2, "ether")),
        "kind": "sell",
        "partiallyFillable": False,
        "from": ADDRESS.lower()
    }
)

if quote_response.status_code == 200:
    quote_data = quote_response.json()
    print("成功获取报价:", quote_data)
    
    # 使用报价更新订单
    order["sellAmount"] = quote_data.get("sellAmount", order["sellAmount"])
    order["buyAmount"] = quote_data.get("buyAmount", order["buyAmount"])
    order["feeAmount"] = quote_data.get("feeAmount", order["feeAmount"])
    if "quoteId" in quote_data:
        order["quoteId"] = quote_data["quoteId"]
else:
    print(f"获取报价失败: {quote_response.status_code}")
    print(quote_response.text)
    # 继续使用默认值

# === 使用CoW API获取订单摘要用于签名 ===
print("正在获取订单摘要用于签名...")
order_digest_response = requests.post(
    "https://api.cow.fi/sepolia/api/v1/orders/digest",
    json=order
)

if order_digest_response.status_code == 200:
    order_digest = order_digest_response.json().get("digest")
    print(f"订单摘要: {order_digest}")
    
    # 使用摘要签名
    order_hash_bytes = binascii.unhexlify(order_digest[2:])  # 移除'0x'前缀
    signed_message = Account.sign_message(
        order_hash_bytes,
        private_key=PRIVATE_KEY
    )
    
    # 添加签名到订单
    order["signature"] = signed_message.signature.hex()
    
    # 提交订单
    print("正在提交订单...")
    submit_response = requests.post(
        "https://api.cow.fi/sepolia/api/v1/orders",
        json=order
    )
    
    print(f"状态码: {submit_response.status_code}")
    print(f"返回内容: {submit_response.text}")
else:
    print(f"获取订单摘要失败: {order_digest_response.status_code}")
    print(order_digest_response.text)

正在获取订单摘要...
获取报价失败: 404
{"errorType":"NoLiquidity","description":"no route found"}
正在获取订单摘要用于签名...
获取订单摘要失败: 404



In [33]:
from web3 import Web3
from eth_account import Account
import json
import requests
import binascii
import eth_utils

# === 配置 ===
PRIVATE_KEY = "6125280318b5f59e2864716ea224dec13c1aeecc7c8747b882c1a4b190e2e2a1"
ADDRESS = Web3.to_checksum_address("0x6722Da2471fd315205ae9d9664bd5171A5ba2Abc")
web3 = Web3(Web3.HTTPProvider("https://ethereum-sepolia.publicnode.com"))

# === Token 合约 ===
WETH = Web3.to_checksum_address("0xfff9976782d46cc05630d1f6ebab18b2324d6b14")
COW = Web3.to_checksum_address("0x0625aFB445C3B6B7B929342a04A22599fd5dBB59")
COWSWAP_CONTRACT = Web3.to_checksum_address("0x9008d19f58aabd9ed0d60971565aa8510560ab41")

# === 构造订单 ===
app_data = "{\"version\":\"0.9.0\",\"metadata\":{}}"
app_data_hash = Web3.keccak(text=app_data).hex()

current_time = int(web3.eth.get_block('latest').timestamp)
valid_to = current_time + 600  # 10分钟后过期
buy_amount_cow = str(165 * 10**18)  # 165个 COW，18位精

# === 使用CowSwap API获取订单摘要 ===
print("正在获取订单摘要...")
quote_response = requests.post(
    "https://api.cow.fi/sepolia/api/v1/quote",
    json={
        "sellToken": WETH,
        "buyToken": COW,
        "receiver": ADDRESS,
        "sellAmountBeforeFee": str(Web3.to_wei(0.5, "ether")),
        "kind": "sell",
        "partiallyFillable": True,
        "from": ADDRESS.lower()
    }
)

if quote_response.status_code == 200:
    quote_data = quote_response.json()
    print("成功获取报价:", quote_data)
    
else:
    print(f"获取报价失败: {quote_response.status_code}")
    print(quote_response.text)
    # 继续使用默认值
order = {
    "sellToken": quote_data["quote"]["sellToken"],
    "buyToken": quote_data["quote"]["buyToken"],
    #"receiver": quote_data["quote"]["receiver"],
    "sellAmount": quote_data["quote"]["sellAmount"],
    "buyAmount": quote_data["quote"]["buyAmount"],
    "validTo": quote_data["quote"]["validTo"],
    "appData": "0x"+ Web3.keccak(text="cow").hex(),
    #"appDataHash": Web3.to_hex(eth_utils.keccak(hexstr=quote_data["quote"]["appData"])),
    "feeAmount": "0",
    "kind": quote_data["quote"]["kind"],
    "partiallyFillable": quote_data["quote"]["partiallyFillable"],
    "sellTokenBalance": quote_data["quote"]["sellTokenBalance"],
    "buyTokenBalance": quote_data["quote"]["buyTokenBalance"],
    #"signingScheme": quote_data["quote"]["signingScheme"],  # "eip712"
    #"quoteId": quote_data["id"],
    #"from": quote_data["from"]
}
print(order)

order["receiver"]= quote_data["quote"]["receiver"]
order["from"]= quote_data["from"]
order["kind"]= quote_data["quote"]["kind"]
order["signingScheme"]= quote_data["quote"]["signingScheme"]
order["appDataHash"] = Web3.to_hex(eth_utils.keccak(hexstr=quote_data["quote"]["appData"]))
order["feeAmount"]: quote_data["quote"]["feeAmount"]


order["signature"] = signed_message.signature.hex()
    # 提交订单
    print("正在提交订单...")
    submit_response = requests.post(
        "https://api.cow.fi/sepolia/api/v1/orders",
        json=order
    )
    
    print(f"状态码: {submit_response.status_code}")
    print(f"返回内容: {submit_response.text}")
else:
    print(f"获取订单摘要失败: {order_digest_response.status_code}")
    print(order_digest_response.text)

正在获取订单摘要...
成功获取报价: {'quote': {'sellToken': '0xfff9976782d46cc05630d1f6ebab18b2324d6b14', 'buyToken': '0x0625afb445c3b6b7b929342a04a22599fd5dbb59', 'receiver': '0x6722da2471fd315205ae9d9664bd5171a5ba2abc', 'sellAmount': '473107794665489160', 'buyAmount': '164428962043613737416', 'validTo': 1746436866, 'appData': '0x0000000000000000000000000000000000000000000000000000000000000000', 'feeAmount': '26892205334510840', 'kind': 'sell', 'partiallyFillable': False, 'sellTokenBalance': 'erc20', 'buyTokenBalance': 'erc20', 'signingScheme': 'eip712'}, 'from': '0x6722da2471fd315205ae9d9664bd5171a5ba2abc', 'expiration': '2025-05-05T08:53:06.054959113Z', 'id': 753410, 'verified': True}
{'sellToken': '0xfff9976782d46cc05630d1f6ebab18b2324d6b14', 'buyToken': '0x0625afb445c3b6b7b929342a04a22599fd5dbb59', 'sellAmount': '473107794665489160', 'buyAmount': '164428962043613737416', 'validTo': 1746436866, 'appData': 'c85ef7d79691fe79573b1a7064c19c1a9819ebdbd1faaab1a8ec92344438aaf4', 'feeAmount': '0', 'kind':

In [50]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_utils import keccak, to_checksum_address
from hexbytes import HexBytes


# === 1. 定义 domain 信息 ===
domain = {
    "name": "Gnosis Protocol",
    "version": "v2",
    "chainId": 11155111,  # Sepolia
    "verifyingContract": "0x9008D19f58AAbD9eD0D60971565AA8510560ab41",
}

# === 2. 定义 order 信息 ===
order = {
    "sellToken": to_checksum_address("0xfff9976782d46cc05630d1f6ebab18b2324d6b14"),
    "buyToken": to_checksum_address("0x0625afb445c3b6b7b929342a04a22599fd5dbb59"),
    "sellAmount": int("473107794665489160"),
    "buyAmount": int("164428962043613737416"),
    "validTo": int(1746436866),
    "appData": HexBytes("0xc85ef7d79691fe79573b1a7064c19c1a9819ebdbd1faaab1a8ec92344438aaf4"),
    "feeAmount": int(0),
    "kind": keccak(text="sell"),
    "partiallyFillable": False,
    "sellTokenBalance": keccak(text="erc20"),
    "buyTokenBalance": keccak(text="erc20"),
    "receiver": to_checksum_address("0x6722Da2471fd315205ae9d9664bd5171A5ba2Abc"),
}

# === 3. 构造 EIP-712 typed data 格式 ===
typed_data = {
    "types": {
        "EIP712Domain": [
            {"name": "name", "type": "string"},
            {"name": "version", "type": "string"},
            {"name": "chainId", "type": "uint256"},
            {"name": "verifyingContract", "type": "address"},
        ],
        "Order": [
            {"name": "sellToken", "type": "address"},
            {"name": "buyToken", "type": "address"},
            {"name": "sellAmount", "type": "uint256"},
            {"name": "buyAmount", "type": "uint256"},
            {"name": "validTo", "type": "uint32"},
            {"name": "appData", "type": "bytes32"},
            {"name": "feeAmount", "type": "uint256"},
            {"name": "kind", "type": "bytes32"},
            {"name": "partiallyFillable", "type": "bool"},
            {"name": "sellTokenBalance", "type": "bytes32"},
            {"name": "buyTokenBalance", "type": "bytes32"},
            {"name": "receiver", "type": "address"},
        ]
    },
    "primaryType": "Order",
    "domain": domain,
    "message": order,
}

# === 4. 计算 order digest ===
# 使用encode_typed_data生成签名哈希
signable_message = encode_typed_data(full_message=typed_data)
# SignableMessage是具有version、header和body的命名元组
# 对于EIP-712，body包含实际的哈希
digest = HexBytes(signable_message.body)

# === 5. 输出 digest ===
print("Order digest:", digest.hex())


account = Account.from_key(PRIVATE_KEY)

# === 1. 构造 Ethereum Signed Message 哈希 ===
eth_sign_message = b"\x19Ethereum Signed Message:\n32" + digest
eth_message_hash = keccak(eth_sign_message)

# === 2. 签名该哈希 ===
signed = Account.sign_hash(eth_message_hash, private_key=PRIVATE_KEY)

# === 3. 输出 signature（可直接提交给 CoW Protocol）===
signature = signed.signature.hex()
print("eth_sign signature:", signature)

Order digest: 58dadd64d509d8be232c2164ef77103c6835e9162c5e0ac10b8b65abd8a96336


AttributeError: type object 'Account' has no attribute 'sign_hash'